In [159]:
import torch


def print_config(config):
    attrs = {}
    attrs.update(config.__dict__)

    for key, value in config.__class__.__dict__.items():
        if not key.startswith('__') and key not in attrs:
            attrs[key] = value

    for key, value in attrs.items():
        print(f"{key}: {value}")


class Config:
    # Model
    model_name = "tabnet.86.repro.stopping.at.70"
    input_dim = 6
    target_dim = 1

    # Device & reproducibility
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 5274

    # Optimization
    optimizer_name = "AdamW"
    lr = 1e-4
    weight_decay = 1e-3

    batch_size = 32  # // grad_accum_steps

    # Training schedule
    num_epochs = 100
    scheduler_name = "default"

    # Regularization
    dropout = 0.0
    drop_path_rate = 0.0
    label_smoothing = 0.0

    # Experiment tracking
    neptune_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
    with_id = ""
    resume = False

config = Config()

In [160]:
import random
import numpy as np


def setup_reproducibility(config):
    random.seed(config.seed)
    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(config.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility(config)

In [161]:
import os

PATH = "/Users/arbaaz/Downloads/break/ds"
inputs_path = PATH+"/X_train.parquet"
targets_path = PATH+"/y_train.parquet"
os.listdir(PATH)

['y_train.parquet',
 'X_train.parquet',
 '.gitignore',
 'X_test.reduced.parquet',
 'y_test.reduced.parquet']

In [212]:
import pandas as pd

def load_parquet(path):
    return pd.read_parquet(path)

inputs_df = load_parquet(inputs_path)
targets_df = load_parquet(targets_path)

In [213]:
inputs_df

value  period
id    time                  
0     0    -0.005564       0
      1     0.003705       0
      2     0.013164       0
      3     0.007151       0
      4    -0.009979       0
...              ...     ...
10000 2134  0.001137       1
      2135  0.003526       1
      2136  0.000687       1
      2137  0.001640       1
      2138  0.001074       1

[23715734 rows x 2 columns]

In [ ]:
targets_df

,structural_breakpoint
id,
0,False
1,False
2,True
3,False
4,False
...,...
9996,False
9997,False
9998,False


In [224]:
from sklearn.model_selection import train_test_split

def split(inputs_df, targets_df, seed):
    targets_df = targets_df.reset_index()

    train_ids, test_ids = train_test_split(
        targets_df['id'],                     
        test_size=0.2,
        random_state=seed,
        stratify=targets_df['structural_breakpoint']
    )

    train_inputs = inputs_df.loc[inputs_df.index.get_level_values('id').isin(train_ids)].copy()
    eval_inputs = inputs_df.loc[inputs_df.index.get_level_values('id').isin(test_ids)].copy()

    train_targets = targets_df[targets_df['id'].isin(train_ids)].set_index('id').copy()
    eval_targets = targets_df[targets_df['id'].isin(test_ids)].set_index('id').copy()
    
    return (
        train_inputs,
        train_targets, 
        eval_inputs,
        eval_targets
    )
    
data = split(inputs_df, targets_df, 1)

In [ ]:
import torch
import torch.nn.functional as F


def preprocess_inputs(inputs_df, dimension=2048):
    inputs = []    
    for id, seq in inputs_df.groupby("id"):
        value = seq["value"].values.astype(np.float32)
        period = seq["period"].values.astype(np.float32)
        input = torch.tensor((value, period)).unsqueeze(0)
        input = F.interpolate(input, size=dimension, mode="nearest-exact").squeeze(0)
        inputs.append(input) # 2, SEQ LEN
    return torch.stack(inputs)

train_inputs = preprocess_inputs(data[0])
eval_inputs = preprocess_inputs(data[2])
train_inputs.shape, eval_inputs.shape

(torch.Size([8000, 2, 2048]), torch.Size([2001, 2, 2048]))

In [230]:
def preprocess_targets(target_df):
    targets = []
    for id, target in target_df.groupby("id"):
        target = target["structural_breakpoint"].values.astype(np.int32)
        targets.append(torch.tensor(target))
    return torch.stack(targets)    

train_targets = preprocess_targets(data[1])
eval_targets = preprocess_targets(data[3])
train_targets.shape, eval_targets.shape

(torch.Size([8000, 1]), torch.Size([2001, 1]))

In [232]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import numpy as np

class SequentialDataset(Dataset):
    def __init__(
        self, 
        inputs, 
        targets,
        mean=None,
        std=None,
        min=None,
        max=None,
        minmax=False,
        zscore=False,
    ):  
        if zscore:
            inputs = self.perform_zscore(inputs, mean, std)
        if minmax:
            inputs = self.perform_minmax(inputs, min, max)
            
        assert len(inputs) == len(targets), "Length Error"
        self.inputs = inputs
        self.targets = targets
            
    def perform_zscore(self, inputs, mean, std):
        for i in range(len(inputs)):
            inputs[i][0] = (inputs[i][0] - mean) / std
        return inputs

    def perform_minmax(self, inputs, min, max):
        for i in range(len(inputs)):
            inputs[i][0] = (inputs[i][0] - min) / (max - min + 1e-8)
        return inputs
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):   
        return self.inputs[index], self.targets[index]

x = train_inputs
mean, std = x.mean(), x.std()
min, max =  x.min(), x.max()
print(min, max, mean, std)

train_ds = SequentialDataset(train_inputs, train_targets, mean, std, zscore=True)
eval_ds = SequentialDataset(eval_inputs, eval_targets, mean, std, zscore=True)

tensor(-2.2848) tensor(412.5000) tensor(0.1335) tensor(0.3484)


In [233]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
train_dl = build_loader(
    config.seed,
    train_ds,
    train=True,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

eval_dl = build_loader(
    config.seed,
    eval_ds,
    train=False,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

In [234]:
for id, (i, t), in enumerate(train_dl):
    
    print(i[0].mean(), i[0].std())
    if id == 10:
        break

tensor(-0.1022) tensor(0.3905)
tensor(-0.0771) tensor(0.4268)
tensor(-0.0563) tensor(0.4533)
tensor(-0.1084) tensor(0.3803)
tensor(-0.0351) tensor(0.4778)
tensor(-0.1084) tensor(0.3827)
tensor(-0.0626) tensor(0.4456)
tensor(-0.0347) tensor(0.4785)
tensor(0.0349) tensor(0.5448)
tensor(-0.0730) tensor(0.4318)
tensor(0.0483) tensor(0.5575)


/Users/arbaaz/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
